# NMT with the help reducing padded sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Making necessary Imports

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor

fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.
fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


In [ ]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

In [ ]:
!pip install -U pip setuptools wheel

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import math
import time
import torch.nn.functional as F
from indicnlp.tokenize import indic_tokenize  
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
spacy_en = spacy.load('en_core_web_sm')


# Reading and Writing auxilary files

In [ ]:
df=pd.read_csv('./drive/MyDrive/train/train.csv')
df=df[['hindi','english']]
train, val=train_test_split(df,test_size=0.02,random_state=42)
train.to_csv('train.csv',index=False)
val.to_csv('val.csv',index=False)
df2=pd.read_csv('hindistatements.csv')
df3=pd.read_csv('train.csv')

with open('actual_test.csv', 'w') as file:
     writer = csv.writer(file)
     writer.writerow(["hindi", "english"])
     for i in range(len(df2)):
         writer.writerow([df2.iloc[i]['hindi'],df3.iloc[i]['english']])
         
df4=pd.read_csv('actual_test.csv')
df4.to_csv('test.csv',index=False)

In [ ]:
def tokenize_hindi(text):

    text=text.replace('.','')
    text=text.replace(',','')
    text=text.replace('  ','')
    text=text.replace('-','')
    text=text.replace('(','')
    text=text.replace(')','')
    text=text.replace(':','')
    text=text.replace('"','')
    text=text.replace('!','')
    text=text.replace('?','')
    text=text.replace('/','')
    text=text.replace('~','')
    text=text.replace('[','')
    text=text.replace('।','')
    text=text.replace('|','')

    
    return [t for t in indic_tokenize.trivial_tokenize(text)]

def tokenize_english(text):

    text=text.replace('.','')
    text=text.replace(',','')
    text=text.replace('  ','')
    text=text.replace('-','')
    text=text.replace('(','')
    text=text.replace(')','')
    text=text.replace(':','')
    text=text.replace('"','')
    text=text.replace('!','')
    text=text.replace('?','')
    text=text.replace('/','')
    text=text.replace('~','')
    text=text.replace('[','')

    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_hindi, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = False,
            include_lengths = True)

TRG = Field(tokenize = tokenize_english, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

fields={'hindi': ('src',SRC),'english':('trg',TRG)}
train_data, valid_data, test_data=TabularDataset.splits(
    path='.',
    train='train.csv',
    validation='val.csv',
    test='test.csv',
    format='csv',
    fields= fields,
)
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 4,
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

cuda


# The Bi-GRU Encoder and Attention architecter

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        embedded = self.dropout(self.embedding(src))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
    
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
       
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

# The GRU decoder Architecture

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):

        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        
        a = a.unsqueeze(1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        weighted = torch.bmm(a, encoder_outputs)
        
        weighted = weighted.permute(1, 0, 2)
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        input = trg[0,:]
        
        mask = self.create_mask(src)
                
        for t in range(1, trg_len):
            
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1) 
            
            input = trg[t] if teacher_force else top1
            
        return outputs

# Model Initialisations

In [ ]:

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(512, 512)
enc = Encoder(len(SRC.vocab), 256, 512, 512, 0.2)
dec = Decoder(len(TRG.vocab), 256, 512, 512, 0.2, attn )

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

#Training and Evaluation

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src_len , trg)
    
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0)
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

# Store and Load the best model parameter values

In [ ]:

best_valid_loss = float('inf')

for epoch in range(5):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, 1)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'padded-model.pt')
    
    print(f'Epoch: {epoch+1:02} ')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

In [ ]:
model.load_state_dict(torch.load('padded-model.pt'))

# Get the translation and prepare answer.txt

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
    
    
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]


    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

file1 = open("answer.txt","w")
for example_idx in range(5000):
    src = vars(test_data.examples[example_idx])['src']
    translation, attention = translate_sentence(src, SRC, TRG, model, device)
    translation=translation[:-1]
    trans=''
    for i in range(len(translation)):
        if translation[i]=="n't" or translation[i][0]=="'" or translation[i][0]=="," or translation[i][0]=="?" or translation[i][0]=="." or translation[i][0]=="!":
            trans=trans+translation[i]
        else:
            trans=trans+' '+translation[i]
    
    file1.write(trans[1:]+'\n')

file1.close()



In [ ]:
file1 = open("answer.txt","r")
lines=file1.readlines()

print(len(lines))
for i in range(4950,5000):
    print(vars(test_data.examples[i])['src'])
    print(lines[i])
